<a href="https://colab.research.google.com/github/AlirezaPNouri/BERTEmbedding/blob/main/BertEmbedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is an optimized version of BERT which uses the BERT embedding instead of Bert classification.
3/16/2022

In [1]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 4.2 MB/s 
     |████████████████████████████████| 67 kB 6.7 MB/s 
     |████████████████████████████████| 6.5 MB 57.8 MB/s 
     |████████████████████████████████| 895 kB 75.8 MB/s 
     |████████████████████████████████| 596 kB 67.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import requests
import numpy as np
import pandas as pd
import string
import pickle

In [24]:
MAX_DATA = 100
MAX_SENTENCE_LENGTH = 500
MIN_SENTENCE_LENGTH = 200
doc_list =[item for item in range(MAX_DATA)]
jump = 3 # jump size for gradient decent
MAX_FEATURE_SIZE = 20 # max size of the gradient descent vector
range_n_clusters = [2, 3, 4, 5, 6] #number of different clustering to compare the performance of the model over different clustering
threshold_cluster_overlapping = 0.7 # to consider two clsuters are similar
stopwords = ['i','me','my','myself','we','our','ours','ourselves','you','your','yours','yourself','yourselves','he','him','his','himself','she','her','hers','herself','it','its','itself','they','them','their','theirs','themselves','what','which','who','whom','this','that','these','those','am','is','are','was','were','be','been','being','have','has','had','having','do','does','did','doing','a','an','the','and','but','if','or','because','as','until','while','of','at','by','for','with','about','against','between','into','through','during','before','after','above','below','to','from','up','down','in','out','on','off','over','under','again','further','then','once','here','there','when','where','why','how','all','any','both','each','few','more','most','other','some','such','no','nor','not','only','own','same','so','than','too','very','s','t','can','will','just','don','should','now', 'of', 'within']


In [ ]:
# download the dataset as a zip file from the git repo
print('Downloading dataset...')

# The URL for the dataset zip file.
url = 'https://raw.githubusercontent.com/AlirezaPNouri/BERTEmbedding/main/5KArticles.csv'
res = requests.get(url, allow_redirects=True)
with open('small_dataset_NYT.csv','wb') as file:
    file.write(res.content)
print('Download is done!')

Download is done!


In [ ]:
# Load the dataset into a pandas dataframe.
df = pd.read_csv("/content/small_dataset_NYT.csv", header= None, skiprows=1)
print('The original size of dataset is {}'.format(df.shape))
df = df.dropna(how='any', axis=0)

# Report the number of sentences.
print('Number of training sentences extracted from dataset is {:,}\n'.format(df.shape[0]))
# Display 10 random rows from the data.
# print(df.sample(10))
df = df[:][[1,2]]
df.columns = ['id', 'content']
df['text_length'] = df['content'].apply(lambda x : len(x.split(' ')))
df = df[df['text_length']>= MIN_SENTENCE_LENGTH]
df['content'] = df['content'].apply(lambda x : ' '.join(x.split(' ')[:MAX_SENTENCE_LENGTH]))
df = df[0:MAX_DATA][['content']]
print('dataset shape is {}'.format(df.shape))
print('Columns are : {}'.format([name for name in df.columns]))
print('The dimension of the dataset is {}'.format(df.shape))

The original size of dataset is (5001, 3)
Number of training sentences extracted from dataset is 5,001

dataset shape is (100, 1)
Columns are : ['content']
The dimension of the dataset is (100, 1)


In [ ]:
#run Bert over documents to generate embedding for each documeny and tokens
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained('bert-base-uncased')
# input_ids = tf.constant(tokenizer.encode("Hello, my dog is cute"))[None, :]  # Batch size 1
# outputs = model(input_ids)
# last_hidden_states = outputs[0]  # The last hidden-state is the first element of the output tuple

# print(tokenizer.tokenize("Hello, my dog is cute"))
# print(last_hidden_states.shape) # the first and the last ones are the cls and sep
Embedding_dict = {}
for id, doc in zip(range(MAX_DATA),df['content']):
  Embedding_dict[id] = model(tf.constant(tokenizer.encode(doc))[None, :])[0]
  print('Document {} is done!'.format(id))
# print(Embedding_dict)

In [ ]:
# create a dictionary out of the embedding for each document and all tokens in the document
# the structure of doc_word_embedding:  {doc0: {'cls': embedding_vec, 'ali': embedding_vec... 'sep': embedding_vec}, doc1: {}} --> {doc#: {word: embedding_vec}}

doc_word_embedding = {}
for doc in range(MAX_DATA):
  tmp_list = dict()
  temp_list = ()
  temp_list = np.array(Embedding_dict[doc][0])
  tmp_list['cls']= temp_list[0]
  for word, embedding in zip(tokenizer.tokenize(df['content'].iloc[doc]), temp_list[1:-1]):
    #filter some unuseful tokens
    if len(word)>2 and word not in string.punctuation:
      tmp_list[word]=embedding
  tmp_list['sep']= temp_list[-1]
  doc_word_embedding[doc] = tmp_list
  print('doc {} is done!'.format(doc))


In [ ]:
#Function to find similarities
from sklearn.metrics.pairwise import cosine_similarity
def find_similar(embedding_dict):
  score_dic = dict()
  word_list = list(embedding_dict.keys())[1:-1] #ignore 'cls' and 'sep' tokens
  for word in word_list:
    temp_list = list()
    for neighbor in word_list:
      temp_list.append(cosine_similarity([ embedding_dict[word]], [embedding_dict[neighbor] ])[0][0])
    score_dic[word] = sorted(temp_list, reverse=True)[1:]# ignore the cosine similarity between each word and itself
  return score_dic

In [ ]:
# create the neighbor similarity vector for each word in each document
# the structure of it is {doc0: {token1: similarity distribution, token2: similarity distribution}, doc1: ...}. be careful the size of each neighbor vector is one less than the number of tokens in each document
word_neighbor_similarity = dict()
for doc in doc_word_embedding.keys():
  word_neighbor_similarity[doc] = find_similar(doc_word_embedding[doc])
  print('doc {} is done'.format(doc))


In [ ]:
print(word_neighbor_similarity[3]['despite'])

[0.64051676, 0.58452755, 0.56511635, 0.5548535, 0.54290926, 0.5135425, 0.51309526, 0.5109051, 0.5035542, 0.49692708, 0.49628136, 0.47123325, 0.4648232, 0.4627166, 0.45677203, 0.45496714, 0.4527735, 0.45257795, 0.44931555, 0.4294933, 0.42622155, 0.42570502, 0.42535168, 0.4253087, 0.42384106, 0.4233408, 0.42186263, 0.4157315, 0.414689, 0.41300115, 0.41061467, 0.40746665, 0.39894292, 0.39807484, 0.39604592, 0.39546376, 0.3857022, 0.38423222, 0.38123444, 0.38118744, 0.38025457, 0.3736773, 0.37283686, 0.37184757, 0.3718049, 0.37087893, 0.37052423, 0.36995998, 0.3692636, 0.36876047, 0.366902, 0.3646148, 0.36326277, 0.36170065, 0.36019766, 0.3591764, 0.35829365, 0.35526678, 0.3551512, 0.35510546, 0.35455137, 0.35448137, 0.35408294, 0.35392505, 0.35386336, 0.35359597, 0.35270667, 0.35210943, 0.34991646, 0.34986964, 0.34955424, 0.34895927, 0.34672856, 0.34666568, 0.34580514, 0.34531197, 0.34316522, 0.3418697, 0.34046656, 0.34012195, 0.34009957, 0.33893055, 0.3388222, 0.3372243, 0.33662128, 0.33

In [ ]:
from transformers.utils.dummy_tf_objects import TF_LXMERT_PRETRAINED_MODEL_ARCHIVE_LIST
##Calculate the gradient descent of the similarity distribution
## MAX_FEATURE_SIZE will cut the rest of the features
gradient_descent_neighbors = dict()
for doc in word_neighbor_similarity.keys():
  temp_dict = dict()

  for word, embedding in word_neighbor_similarity[doc].items():
    vec_size = len(embedding)
    vec_size -= vec_size%jump
    embedding = embedding[0:vec_size]
    new_list = [embedding[x:x+jump] for x in range(0, vec_size, jump)]
    tmp_list = list()
    for ele in new_list:
      m1,b1 = np.polyfit(np.arange(0, jump), ele, 1)
      tmp_list.append(m1)
    temp_dict[word] = tmp_list
  gradient_descent_neighbors[doc] = temp_dict
  print('Document {} is done!'.format(doc))

  

In [ ]:
from numpy import median, array, exp
#sigmoid function
def sigmoid(vec_):
  z = array(vec_)
  z = z/np.linalg.norm(z)
  g = 1 / (1 + exp(-z))
  return g

In [ ]:
for doc in gradient_descent_neighbors.keys():
  for word, gradient_vec in gradient_descent_neighbors[doc].items():
    gradient_descent_neighbors[doc][word] =sigmoid(gradient_descent_neighbors[doc][word][0:MAX_FEATURE_SIZE])# just take top features and apply sigmoid function

In [ ]:
from sklearn.cluster import KMeans
# clustering the tokens in each document. It creates differnt number of clustering
all_clusters = dict()
all_centroid = dict()
for doc, featurs in gradient_descent_neighbors.items():
  clusters_per_doc = dict()
  centroids_per_doc = dict()
  for n_clusters in range_n_clusters:
    clusterer = KMeans(n_clusters=n_clusters, random_state=10, max_iter=100,)
    clusters_per_doc[n_clusters] = clusterer.fit_predict(list(featurs.values()))
    centroids_per_doc[n_clusters] = clusterer.cluster_centers_
  all_clusters[doc] = clusters_per_doc
  all_centroid[doc] = centroids_per_doc
  print('Document {} is done!'.format(doc))
     

In [ ]:
#create a total clusters sorted dictionary
## the structure of total_docs_clusters  is {doc0: {number_of_clusters { cluster_number: {token0: closeness_to_centroid, token1: closeness_to_centroid ...} }}}
total_docs_clusters = dict()
for doc in all_clusters.keys():
  temp_cluster = all_clusters[doc]
  dic_for_clustering = dict()
  for n_cluster in range_n_clusters:
    temp_dict = dict()
    for la in range(n_cluster):
      temp_list = dict()
      for label_, value_ in zip(temp_cluster[n_cluster],gradient_descent_neighbors[doc].items()):
        if label_ == la:
          temp_list[value_[0]] = np.sum(np.square(value_[1] - all_centroid[doc][n_cluster][la]))
      temp_dict[la]=temp_list
    dic_for_clustering[n_cluster] = temp_dict
  total_docs_clusters[doc] = dic_for_clustering
  
    
      

In [12]:


# pickle.dump(df, open("df.p","wb"))
# pickle.dump(Embedding_dict, open("Embedding_dict.p", "wb"))
# pickle.dump(doc_word_embedding, open("doc_word_embedding.p","wb"))
# pickle.dump(word_neighbor_similarity, open("word_neighbor_similarity.p","wb"))
# pickle.dump(gradient_descent_neighbors, open("gradient_descent_neighbors.p","wb"))
# pickle.dump(total_docs_clusters, open("total_docs_clusters.p","wb"))
# pickle.dump(all_clusters, open("all_clusters.p","wb"))
# pickle.dump(all_centroid, open("all_centroid.p","wb"))

# df = pickle.load(open("df.p", "rb"))
# Embedding_dict = pickle.load(open("Embedding_dict.p", "rb"))
# doc_word_embedding = pickle.load(open("doc_word_embedding.p", "rb"))
# word_neighbor_similarity = pickle.load(open("word_neighbor_similarity.p", "rb"))
total_docs_clusters = pickle.load(open("total_docs_clusters.p", "rb"))
# all_clusters = pickle.load(open("all_clusters.p", "rb"))
# all_centroid = pickle.load(open("all_centroid.p", "rb"))

In [9]:
print(total_docs_clusters[0])

{2: {0: {'hurrying': 0.014334454281205427, 'through': 0.024271209334057547, 'the': 0.016466063089275618, 'veterans': 0.01057808828276694, 'guard': 0.010406854711459945, 'new': 0.014575321405436081, 'yelled': 0.00922522563525655, 'warm': 0.00998651367828293, 'huh': 0.014299272795611271, 'grin': 0.01196944871267222, 'room': 0.01257380783044098, 'wild': 0.005893532163674162, 'philadelphia': 0.02356714565081841, 'eagles': 0.008064060730212059, 'cold': 0.008152779266458863, 'qualified': 0.008441635526562834, 'but': 0.023569344884982423, 'until': 0.010136276509536921, 'time': 0.00875520217791407, 'teams': 0.01526858105802071, 'minnesota': 0.016405918346635427, 'metro': 0.011629377955285014, '##dome': 0.024294921778461667, 'washington': 0.0074125138276196325, 'and': 0.034586827581737624, 'green': 0.016517300514080895, 'being': 0.010200571798814395, 'fired': 0.01799521918403016, 'going': 0.005585028947087353, 'right': 0.02918924862791166, 'john': 0.015623067128077624, 'this': 0.007162647127223

In [14]:
print(total_docs_clusters[0])

{2: {0: {'hurrying': 0.3946213922650285, 'through': 0.6897583799973143, 'the': 0.45793346942616076, 'veterans': 0.28305151460351713, 'guard': 0.27796561280102855, 'new': 0.40177551824590074, 'yelled': 0.24286940238700064, 'warm': 0.2654808342498523, 'huh': 0.39357644775075007, 'grin': 0.32437707047643544, 'room': 0.34232747064008, 'wild': 0.14391295533561516, 'philadelphia': 0.6688466000915336, 'eagles': 0.20838100949490537, 'cold': 0.21101608718421438, 'qualified': 0.21959556469384972, 'but': 0.6689119207463181, 'until': 0.26992902187805334, 'time': 0.228908979026886, 'teams': 0.42236640173773354, 'minnesota': 0.4561470775833162, 'metro': 0.3142764432176112, '##dome': 0.690462676256604, 'washington': 0.1890290591675965, 'and': 0.9961481903552376, 'green': 0.45945530015820124, 'being': 0.27183869137295386, 'fired': 0.5033517695832913, 'going': 0.13474993284337633, 'right': 0.8358317505309376, 'john': 0.4328951860695586, 'this': 0.18160763186086296, 'wasn': 0.2640789858474042, 'nice': 0

In [10]:
# this function will receive a vector and the min and max of that vector and will return the normalized vec 
def normalize_vec(vec, min_, max_):
    range_val = (max_ - min_)+0.0000001
    vec = [item/range_val for item in vec]
    min_vec = min(vec)
    vec = [item - min_vec for item in vec]
    return vec


# this function will receive a value of a vector, the min and the max of the vector and returns the normalized value
def normalize_value(item, min_, max_):
    range_val = (max_ - min_)+0.0000001
    vec = (item-min_)/range_val
    return vec

In [ ]:
# normalize the similarity in each cluster. It goes and find the min and max values in each clustering and make those similarity between [0,1] (the similarity here is euclidean distance between each token and the centroid)
for doc in total_docs_clusters.keys():
  temp_doc = total_docs_clusters[doc]
  for cluster_rank in temp_doc.keys():
    temp_clustering = temp_doc[cluster_rank]
    min_sim = 1
    max_sim = 0
    for ind in temp_clustering.keys():
      for ele, val_ in temp_clustering[ind].items():
        min_sim = min_sim if min_sim<val_ else val_
        max_sim = max_sim if val_<max_sim else val_ 

    for ins in temp_clustering.keys():
      for el in total_docs_clusters[doc][cluster_rank][ins].keys():
        total_docs_clusters[doc][cluster_rank][ins][el] = normalize_value(total_docs_clusters[doc][cluster_rank][ins][el], min_sim, max_sim)

In [30]:
#remove stopwords
total_docs_clusters_wo_stopwords = dict()
for doc in total_docs_clusters.keys():
  temp_doc = {}
  for cluster_rank in total_docs_clusters[doc].keys():
    temp_clustering = {}
    for ins in total_docs_clusters[doc][cluster_rank].keys():
      temp_cluster = {}
      for items in total_docs_clusters[doc][cluster_rank][ins].items():
        if items[0] not in stopwords:
          temp_cluster[items[0]] = items[1]
      temp_clustering[ins] = temp_cluster
    temp_doc[cluster_rank] = temp_clustering
  total_docs_clusters_wo_stopwords[doc] = temp_doc

In [33]:
# Jaccard index function
def jaccard_similarity(text1_list, text2_list):
  a = set(text1_list)
  b = set(text2_list)
  return len(a.intersection(b))/len(a.union(b))

In [46]:
total_doc_word_groups = dict()
for doc in total_docs_clusters_wo_stopwords.keys():
  docs_clusters = total_docs_clusters_wo_stopwords[doc]
  # print(docs_clusters)
  temp_list_word_groups = list()
  for cluster_num in list(docs_clusters.keys())[:-1]:
    for cluster in range(cluster_num):
      for next_cluster in range(cluster_num+1):
        cl1 = list()
        cl1_w_sim = dict()
        cl2 = list()
        cl2_w_sim = dict()
        joint_list = list()
        for item in docs_clusters[cluster_num][cluster].items():
          cl1.append(item[0])
          cl1_w_sim[item[0]] =  item[1]
        for another_item in docs_clusters[cluster_num+1][next_cluster].items():
          cl2.append(another_item[0])
          cl2_w_sim[another_item[0]] = another_item[1]
        # print(' ({}, {}, {}) --> '.format(cluster_num, cluster, next_cluster), jaccard_similarity(cl1, cl2))
        jac_val = jaccard_similarity(cl1, cl2)
        if jac_val > threshold_cluster_overlapping:
          jnt_list = list(set(cl1).intersection(cl2))
          dict_vocab = dict()
          for el in jnt_list:
            dict_vocab[el] = max(cl1_w_sim[el], cl2_w_sim[el])

          temp_list_word_groups.append([dict_vocab ,jac_val])
  

  total_doc_word_groups[doc] = temp_list_word_groups
  # the structure of total_doc_word_group is {doc0: [ [ {word1: normalized_normalized_distance_to_centroid, ....}, similarity between two clusters] [{word1: normalized_distance_to_centroid, ....}, similarity_between_two_clusters]... ], doc1: ...}

In [47]:
#change the distance to centroid to the frequency [0, 1] --> [10, 1] 
for doc in total_doc_word_groups.keys():
  temp_list = list()
  for word_list in total_doc_word_groups[doc]:
    temp_dict = dict()
    for ele in word_list[0].keys():
      temp_dict[ele] =int(9* (1-word_list[0][ele]))+1
    temp_list.append(temp_dict)
  total_doc_word_groups[doc] = temp_list
# the new structure of total_doc_word_groups is {doc0: [{groupOnewords}, grouptwowords], doc1}   , grouponewords = {word1:frequency, word2:ferquency}

In [45]:
print(total_doc_word_groups[0])

[{'locker': 8, 'year': 8, 'tom': 9, 'somewhat': 9, 'tunnel': 9, 'played': 8, 'four': 9, 'talked': 9, 'guys': 6, 'philadelphia': 4, 'never': 9, 'bob': 8, 'anaheim': 7, 'win': 7, 'yelled': 8, 'stadium': 9, 'yesterday': 9, 'warm': 7, 'jim': 7, 'temperatures': 7, 'behold': 7, 'chicago': 8, 'disappeared': 9, 'playoff': 8, 'jones': 4, '##ly': 9, '##zle': 7, 'said': 8, 'get': 8, 'warmth': 6, 'tournament': 9, 'grin': 8, 'league': 9, 'weather': 8, 'washington': 8, 'moved': 7, '##dome': 4, 'eric': 9, 'calmly': 9, 'entire': 7, 'saying': 8, 'celebrate': 9, 'game': 8, '##iously': 8, 'bay': 8, 'marrow': 7, 'qualified': 8, 'quarterback': 6, 'detroit': 9, 'like': 9, '##iv': 7, 'games': 9, 'card': 9, 'ago': 9, 'wasn': 7, 'accomplished': 9}, {'ted': 8, 'franchise': 8, 'bone': 9, 'afternoon': 10, 'super': 9, 'described': 9, 'playing': 9, 'decades': 9, 'rams': 9, 'plus': 8, 'might': 8, 'anyone': 9, 'los': 9, 'francisco': 9, 'three': 9, 'eye': 9, 'starting': 9, 'cleveland': 8, 'raw': 8, 'course': 9, 'sunny

In [61]:
# doc_matrix is a dictionary of each doc and other doc similarity to this doc based on BERT cls vector             doc_embedding ={doc0: {1: int(9*euclidian_distance +1 ), 2:int(9*euclidian_distance +1 )}, doc1}  
doc_embedding = dict()
for first_doc in doc_word_embedding.keys():
  temp_dict = dict()
  for second_doc in doc_word_embedding.keys(): 
    if first_doc != second_doc:
      temp_dict[second_doc] =int(9*np.square(sum(doc_word_embedding[first_doc]['cls']-doc_word_embedding[second_doc]['cls']))+1)
  doc_embedding[first_doc] = temp_dict
print(doc_embedding)

{0: {1: 4, 2: 4, 3: 1, 4: 14, 5: 7, 6: 3, 7: 36, 8: 1, 9: 1, 10: 8, 11: 6, 12: 13, 13: 2, 14: 1, 15: 3, 16: 1, 17: 1, 18: 10, 19: 4, 20: 10, 21: 1, 22: 4, 23: 1, 24: 1, 25: 6, 26: 11, 27: 1, 28: 21, 29: 3, 30: 1, 31: 25, 32: 11, 33: 5, 34: 7, 35: 3, 36: 20, 37: 1, 38: 2, 39: 19, 40: 3, 41: 2, 42: 1, 43: 2, 44: 1, 45: 1, 46: 1, 47: 8, 48: 3, 49: 7, 50: 3, 51: 1, 52: 15, 53: 1, 54: 3, 55: 1, 56: 1, 57: 8, 58: 3, 59: 1, 60: 3, 61: 1, 62: 2, 63: 4, 64: 1, 65: 5, 66: 2, 67: 4, 68: 2, 69: 2, 70: 3, 71: 1, 72: 6, 73: 1, 74: 1, 75: 1, 76: 1, 77: 3, 78: 12, 79: 1, 80: 1, 81: 6, 82: 11, 83: 2, 84: 1, 85: 8, 86: 7, 87: 1, 88: 1, 89: 1, 90: 6, 91: 12, 92: 54, 93: 2, 94: 1, 95: 3, 96: 1, 97: 1, 98: 3, 99: 6}, 1: {0: 4, 2: 14, 3: 9, 4: 3, 5: 1, 6: 1, 7: 63, 8: 2, 9: 2, 10: 1, 11: 1, 12: 30, 13: 1, 14: 4, 15: 1, 16: 2, 17: 6, 18: 26, 19: 1, 20: 26, 21: 5, 22: 15, 23: 7, 24: 4, 25: 20, 26: 2, 27: 4, 28: 7, 29: 14, 30: 5, 31: 48, 32: 27, 33: 1, 34: 1, 35: 1, 36: 7, 37: 8, 38: 1, 39: 39, 40: 1, 41: 10, 

In [65]:
#calculate the similarity between two documents by using jaccard index of all word groups in each cluster by considering the frequency of those words
# the structure of total_clustering_w_frequency is {doc0:[[word_groups0],[word_groups1], ...], doc1: }
total_clustering_w_frequency = dict()
for doc in total_doc_word_groups.keys():
  list_word_cloud = list()
  for word_group in total_doc_word_groups[doc]:
    temp_list = list()
    for word in word_group.items():
      for ind in range(1, word[1]+1):
        temp_list.append(word[0])
    list_word_cloud.append(temp_list)
  total_clustering_w_frequency[doc] = list_word_cloud




  # print('document {} clusters is {}'.format(doc, list_word_cloud))


In [89]:
#find similar documents
def similar_doc(doc_one, doc_two):
  list_sim = list()
  for clu_1 in doc_one:
    total_jac = 0
    for clu_2 in doc_two:
      jac_val = jaccard_similarity(clu_1, clu_2)
      total_jac += jac_val
    # total_jac /= len(doc_two)
    list_sim.append(total_jac)
  return sum(list_sim)
  # return sum(list_sim)/(len(doc_one)*len(doc_two))   #with normalizing based on the number of word groups

In [90]:
print(similar_doc(total_clustering_w_frequency[0], total_clustering_w_frequency[1]))

0.8263410079850253


In [102]:
#create a matrix of document similarity based on the jaccard index of the similar words in each word group in each document
doc_matrix=dict()
for first_doc in total_clustering_w_frequency.keys():
  temp_dict = dict()
  for second_doc in total_clustering_w_frequency.keys():
    if first_doc != second_doc:
      temp_dict[second_doc] = similar_doc(total_clustering_w_frequency[first_doc], total_clustering_w_frequency[second_doc])
  doc_matrix[first_doc] = temp_dict

print(doc_matrix)
      

#normalize the weight and invert it. if an index is high that means they are similar so the weight must be low

for doc in doc_matrix.keys():
  min_ = 10000
  max_ = -10000
  for ele in doc_matrix[doc].items():
    min_ = min_ if min_<ele[1] else ele[1]
    max_ = max_ if max_> ele[1] else ele[1]
  for ele in doc_matrix[doc].keys():
    doc_matrix[doc][ele] = int(9*(1-normalize_value(doc_matrix[doc][ele], min_, max_))+1)

print(doc_matrix)


{0: {1: 0.8263410079850253, 2: 1.3855352269058099, 3: 0.40214983106808916, 4: 0.6319544120532895, 5: 1.3196332103099886, 6: 0.5851713194170673, 7: 0.8170335204812698, 8: 0.5643801512282562, 9: 0.7153586547993172, 10: 1.0600565215478424, 11: 0.4908179776974525, 12: 0.9561693842739779, 13: 0.666920590910607, 14: 0.5367893193759367, 15: 0.5421530404778326, 16: 0.9301840075289242, 17: 0.5336861253501889, 18: 0.16303676331734046, 19: 0.48941398615416215, 20: 0.11362974962812777, 21: 0.37642660187947685, 22: 0.33714592165591156, 23: 0.4168041007347868, 24: 0.7563509266640143, 25: 0.3852798255900228, 26: 0.8160008125269169, 27: 0.18310722925054973, 28: 0.4883199065244898, 29: 0.3470334018057456, 30: 1.066103892234431, 31: 0.6106567352954874, 32: 0.39063104568969864, 33: 0.6211599875648455, 34: 0.5191074435962817, 35: 0.48267999719207516, 36: 1.0298999483244875, 37: 0.4884437909802653, 38: 0.6411506257524064, 39: 0.2939178968665161, 40: 0.6190736902455, 41: 0.419640105702187, 42: 0.23187943313

In [103]:
# A*Search algorithm

from collections import deque

class Graph:

    def __init__(self,heuristic, adjacency_list, start_node, stop_node):
        self.adjacency_list = adjacency_list
        self.H = dict()
        self.start_node = start_node
        self.stop_node = stop_node
        for ele in heuristic[stop_node]:
          self.H[ele[0]] = ele[1]
        # self.H = {0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1, 22: 1, 23: 1, 24: 1, 25: 1, 26: 1, 27: 1, 28: 1, 29: 1, 30: 1, 31: 1, 32: 1, 33: 1, 34: 1, 35: 1, 36: 1, 37: 1, 38: 1, 39: 1, 40: 1, 41: 1, 42: 1, 43: 1, 44: 1, 45: 1, 46: 1, 47: 1, 48: 1, 49: 1, 50: 1, 51: 1, 52: 1, 53: 1, 54: 1, 55: 1, 56: 1, 57: 1, 58: 1, 59: 1, 60: 1, 61: 1, 62: 1, 63: 1, 64: 1, 65: 1, 66: 1, 67: 1, 68: 1, 69: 1, 70: 1, 71: 1, 72: 1, 73: 1, 74: 1, 75: 1, 76: 1, 77: 1, 78: 1, 79: 1, 80: 1, 81: 1, 82: 1, 83: 1, 84: 1, 85: 1, 86: 1, 87: 1, 88: 1, 89: 1, 90: 1, 91: 1, 92: 1, 93: 1, 94: 1, 95: 1, 96: 1, 97: 1, 98: 1, 99: 1}
      
    # def get_neighbors(self, v):
    #     return self.adjacency_list[v]
    # # heuristic function with equal values for all nodes
    
    # def h(self, n):
    #   # H = {0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1, 22: 1, 23: 1, 24: 1, 25: 1, 26: 1, 27: 1, 28: 1, 29: 1, 30: 1, 31: 1, 32: 1, 33: 1, 34: 1, 35: 1, 36: 1, 37: 1, 38: 1, 39: 1, 40: 1, 41: 1, 42: 1, 43: 1, 44: 1, 45: 1, 46: 1, 47: 1, 48: 1, 49: 1, 50: 1, 51: 1, 52: 1, 53: 1, 54: 1, 55: 1, 56: 1, 57: 1, 58: 1, 59: 1, 60: 1, 61: 1, 62: 1, 63: 1, 64: 1, 65: 1, 66: 1, 67: 1, 68: 1, 69: 1, 70: 1, 71: 1, 72: 1, 73: 1, 74: 1, 75: 1, 76: 1, 77: 1, 78: 1, 79: 1, 80: 1, 81: 1, 82: 1, 83: 1, 84: 1, 85: 1, 86: 1, 87: 1, 88: 1, 89: 1, 90: 1, 91: 1, 92: 1, 93: 1, 94: 1, 95: 1, 96: 1, 97: 1, 98: 1, 99: 1}
    #   return self.H[n]
    def a_star_algorithm(self):
        # open_list is a list of nodes which have been visited, but who's neighbors
        # haven't all been inspected, starts off with the start node
        # closed_list is a list of nodes which have been visited
        # and who's neighbors have been inspected
        open_list = set([self.start_node])
        closed_list = set([])
        # g contains current distances from start_node to all other nodes
        # the default value (if it's not found in the map) is +infinity
        g = {}
        g[self.start_node] = 0
        # parents contains an adjacency map of all nodes
        parents = {}
        parents[self.start_node] = self.start_node
        while len(open_list) > 0:
            n = None
            # find a node with the lowest value of f() - evaluation function
            for v in open_list:
                if n == None or g[v] + self.H[v] < g[n] + self.H[n]:
                # if n == None or g[v] + self.H[v] < g[n] + self.H[n]:
                    n = v;
            if n == None:
                print('Path does not exist!')
                return None
            # if the current node is the stop_node
            # then we begin reconstructin the path from it to the start_node
            if n == self.stop_node:
                reconst_path = []
                while parents[n] != n:
                    reconst_path.append(n)
                    n = parents[n]
                reconst_path.append(self.start_node)
                reconst_path.reverse()
                print('Path found: {}'.format(reconst_path))
                return reconst_path
            # for all neighbors of the current node do
            for (m, weight) in self.adjacency_list[n]:
                # if the current node isn't in both open_list and closed_list
                # add it to open_list and note n as it's parent
                avr = 0
                sam = 0
                for t in heuristic[n]:
                    avr += t[1]
                    if t[0] == m:
                      sam = t[1]
                if sam > avr/2:
                    continue
                if m not in open_list and m not in closed_list:
                    open_list.add(m)
                    parents[m] = n
                    g[m] = g[n] + weight
                # otherwise, check if it's quicker to first visit n, then m
                # and if it is, update parent data and g data
                # and if the node was in the closed_list, move it to open_list
                else:
                    if g[m] > g[n] + weight:
                        g[m] = g[n] + weight
                        parents[m] = n
                        if m in closed_list:
                            closed_list.remove(m)
                            open_list.add(m)
            # remove n from the open_list, and add it to closed_list
            # because all of his neighbors were inspected
            open_list.remove(n)
            closed_list.add(n)

        print('Path does not exist!')
        return None

In [105]:
#create the adjacancy matrix from the euclidean distanse of the doc embedding
#heuristic

heuristic = dict()
for first_doc in doc_embedding.keys():
  temp_list = list()
  for second_doc in doc_embedding.keys():
    if first_doc != second_doc:
      temp_list.append((second_doc, doc_embedding[first_doc][second_doc]))
    else:
      temp_list.append((second_doc, 0))
  heuristic[first_doc] = temp_list
print('heuristic is ', heuristic)


# the weights
doc_weights = dict()
for first_doc in doc_matrix.keys():
  temp_dict = list()
  for second_doc in doc_matrix.keys():
    if first_doc != second_doc:
      temp_dict.append((second_doc, doc_matrix[first_doc][second_doc]))
  doc_weights[first_doc] = temp_dict
print('weights is ', doc_weights)



# print(list_added_nodes) 
# print(adjacency_list)
graph1 = Graph(heuristic, doc_weights, 8, 51)
graph1.a_star_algorithm()


heuristic is  {0: [(0, 0), (1, 4), (2, 4), (3, 1), (4, 14), (5, 7), (6, 3), (7, 36), (8, 1), (9, 1), (10, 8), (11, 6), (12, 13), (13, 2), (14, 1), (15, 3), (16, 1), (17, 1), (18, 10), (19, 4), (20, 10), (21, 1), (22, 4), (23, 1), (24, 1), (25, 6), (26, 11), (27, 1), (28, 21), (29, 3), (30, 1), (31, 25), (32, 11), (33, 5), (34, 7), (35, 3), (36, 20), (37, 1), (38, 2), (39, 19), (40, 3), (41, 2), (42, 1), (43, 2), (44, 1), (45, 1), (46, 1), (47, 8), (48, 3), (49, 7), (50, 3), (51, 1), (52, 15), (53, 1), (54, 3), (55, 1), (56, 1), (57, 8), (58, 3), (59, 1), (60, 3), (61, 1), (62, 2), (63, 4), (64, 1), (65, 5), (66, 2), (67, 4), (68, 2), (69, 2), (70, 3), (71, 1), (72, 6), (73, 1), (74, 1), (75, 1), (76, 1), (77, 3), (78, 12), (79, 1), (80, 1), (81, 6), (82, 11), (83, 2), (84, 1), (85, 8), (86, 7), (87, 1), (88, 1), (89, 1), (90, 6), (91, 12), (92, 54), (93, 2), (94, 1), (95, 3), (96, 1), (97, 1), (98, 3), (99, 6)], 1: [(0, 4), (1, 0), (2, 14), (3, 9), (4, 3), (5, 1), (6, 1), (7, 63), (8, 

[8, 2, 51]